# Compliance

## Create read_data and read_metadata functions

In [ ]:
# Set up commonly-used libraries
# autoreload is good for debugging but causes problems with isinstance()
# %load_ext autoreload
# %autoreload 2

from matplotlib import pyplot as plt
import numpy as np

from read_data_metadata import read_data, read_metadata

## Validate noise removal at AlpArray station A401A

In [ ]:
from tiskitpy import CleanRotator, DataCleaner, ResponseFunctions
from obspy.clients.fdsn import Client
from tiskitpy import SpectralDensity

# Declare parameters for AlpArray expt (note: A401A only recorded data from 2017-12-10 on) 
nslc, st, et, server = "Z3.A401A.*.*", "2018-01-01T00", "2018-01-03T00", "RESIF"

# Read data and metadata
stream = read_data(server, nslc, st, et)
inv = read_metadata(server, nslc, st, et)

sds = {}
# Calculate SpectralDensity, look at PSDs and coherences
print('Original data...')
sds['orig'] = SpectralDensity.from_stream(stream, window_s=2000., inv=inv)
# sds['orig'].plot()
# sds['orig'].plot_coherences()

# Rotate vertical to minimize tilt noise (takes a long time with non-decimated data)
print('Simple rotation...')
stream_rot = CleanRotator(stream).apply(stream)
sds['rot'] = SpectralDensity.from_stream(stream_rot, window_s=2000., inv=inv)

# Try transfer-function based cleaning on residual
print('Transfer function based cleaning...')
stream_rot_clean = DataCleaner(stream_rot, ['*1', '*2']).apply(stream_rot)
sds['rot_clean'] = SpectralDensity.from_stream(stream_rot_clean, window_s=2000., inv=inv)
print('Done')

# SpectralDensity.plots(list(sds.values()), sds_names=list(sds.keys()), channel='BHZ')
SpectralDensity.plots_coherences(list(sds.values()), sds_names=list(sds.keys()), channel_pair=('BDH', 'BHZ'))


## Plot "Compliance Stack" and use it to correct the pressure gauge gain

In [ ]:
from tiskitpy import plot_compliance_stack

station = nslc.split('.')[1]
water_depth = -inv.select(station=station)[0][0].elevation
plot_compliance_stack(sds['orig'],'*HZ', '*DH', water_depth)

## Calculate and plot compliance

In [ ]:
from tiskitpy import ResponseFunctions

rfs = ResponseFunctions(sds['orig'], "*DH", ["*HZ"], "output")
_ = rfs.plot()

In [ ]:
from tiskitpy import Compliance

print(f'Station {station}, {water_depth=}')
ncompl = Compliance.from_response_functions(rfs, water_depth)
_ = ncompl.plot()

### Create a 1D earth model and compare its compliance

In [ ]:
# from tiskitpy.compliance import EarthModel1D

# Based on Dannowski et al. [2020]
earth_model = EarthModel1D([[350., 2000., 2500., 500.],    # Sediments
                            [2000., 3100., 3000., 1300.],  # Uppper crustal rocks
                            [4000., 3100., 6800., 3700.],  # Lower crustal rocks
                            [9999., 3300., 8200., 5000.]   # Deep crust
                           ])
print(earth_model)
earth_model.plot()    # pretty basic for now

# Plot measured versus calculated compliance
f = np.arange(0.003, 0.03, 0.001)
# ncompl_calc = earth_model.calc_ncompl(f, water_depth)
ncompl_calc = Compliance.from_earth_model_1D(water_depth, f, earth_model)
fig, ax = plt.subplots()
ax.loglog(f, np.abs(ncompl_calc.values), c='b', label='model')
ax.errorbar(ncompl.freqs,
            y=np.abs(ncompl.values),
            yerr=np.abs(ncompl.uncertainties),
           fmt='none', c='r', label='measured')
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Normalized compliance (1/Pa)')
ax.legend()
plt.show()

## Exercises

- Try to adjust the model to better fit the data.  What works best?
- Try to adjust the pressure gauge response to better fit the 1/(rho*H) curve
